<a href="https://colab.research.google.com/github/shinnew9/Apziva-Projects/blob/main/Project3-PotentialTalents/Steps2_LambdaRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Open CSV

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv')
tfidf_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/tfidf_df.csv')
w2v_cbow_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/w2v_cbow_df.csv')
w2v_sg_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/w2v_cbow_df.csv')
glove_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/glove_df.csv')
fasttext_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/fasttext_df.csv')
stb_df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/stb_df.csv')
w2v_sg_df

,Unnamed: 0,id,job_title,location,connection,fit,cosine_similarity_w2v_cbow,rank_w2v_cbow
0,5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,1
1,59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,2
2,48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,3
3,35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,4
4,23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN,1.000000,5
...,...,...,...,...,...,...,...,...
99,90,91,Lead Official at Western Illinois University,Greater Chicago Area,39,NaN,-0.189587,100
100,33,34,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,-0.189587,101
101,34,35,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,-0.189587,102
102,22,23,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,-0.189587,103


In [ ]:
fasttext_df

,id,job_title,location,connection,fit,cosine_similarity_ft,rank_ft
0,53,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN,0.983347,1
1,26,HR Senior Specialist,San Francisco Bay Area,500+,NaN,0.983347,2
2,88,Human Resources Management Major,"Milpitas, California",18,NaN,0.983347,3
3,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,NaN,0.983347,4
4,86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,NaN,0.983347,5
...,...,...,...,...,...,...,...
99,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.413387,100
100,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.413387,101
101,13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,NaN,0.413387,102
102,34,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,0.413387,103


In [ ]:
# print("Dataset check:\n", fasttext_df)
print("Are there NaN values?", fasttext_df.isna().sum())
fasttext_df['fit'] = fasttext_df['fit'].fillna(0.5)

Are there NaN values? id                        0
job_title                 0
location                  0
connection                0
fit                     104
cosine_similarity_ft      0
rank_ft                   0
dtype: int64


### LambdaRank

1. Cosine Similarity as fit

In [ ]:
fasttext_df.fit = fasttext_df.cosine_similarity_ft

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
# Step 1: Normalize Fit Scores**
fasttext_df['normalized_fit'] = (fasttext_df['fit'] - fasttext_df['fit'].min()) / (fasttext_df['fit'].max() - fasttext_df['fit'].min())
fasttext_df

,id,job_title,location,connection,fit,cosine_similarity_ft,rank_ft,normalized_fit
0,53,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,0.983347,0.983347,1,1.0
1,26,HR Senior Specialist,San Francisco Bay Area,500+,0.983347,0.983347,2,1.0
2,88,Human Resources Management Major,"Milpitas, California",18,0.983347,0.983347,3,1.0
3,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,0.983347,0.983347,4,1.0
4,86,Information Systems Specialist and Programmer ...,"Gaithersburg, Maryland",4,0.983347,0.983347,5,1.0
...,...,...,...,...,...,...,...,...
99,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.413387,0.413387,100,0.0
100,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.413387,0.413387,101,0.0
101,13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,0.413387,0.413387,102,0.0
102,34,People Development Coordinator at Ryan,"Denton, Texas",500+,0.413387,0.413387,103,0.0


In [ ]:
# Step 2: Generate Pairwise Data**
pairwise_data = []
for _ in range(1000):  # Generate 1000 random pairs
    a, b = random.sample(range(len(fasttext_df)), 2)
    fit_a, fit_b = fasttext_df.loc[a, 'normalized_fit'], fasttext_df.loc[b, 'normalized_fit']
    label = 1 if fit_a > fit_b else 0  # Convert fit scores into 1 or 0
    pairwise_data.append((a, b, label))

# Convert to tensor format => for RankNet
pairwise_data = torch.tensor(pairwise_data, dtype=torch.float32)

# in Lambda, we use listwise, not pairwise

In [ ]:
# Step 3: Define LamdaRank Model
class LambdaRank(nn.Module):
  def __init__(self, input_size):
    super(LambdaRank, self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 1)  # No sigmoid
    self.activation = nn.LeakyReLU(0.01)

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    return self.fc3(x)  # No Sigmoid

In [ ]:
# Step 4: Initialize Model
input_size = 1   # Using fit scores
lambdarank = LambdaRank(input_size)

In [ ]:
# Apply Xavier Weight Initialization
def init_weights(m):
  if isinstance(m, nn.Linear):
    torch.nn.init.xavier_uniform_(m.weight)
    if m.bias is not None:
      m.bias.data.fill_(0)
lambdarank.apply(init_weights)

LambdaRank(
  (fc1): Linear(in_features=1, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (activation): LeakyReLU(negative_slope=0.01)
)

In [ ]:
# Ensure Gradients are Updating
for param in lambdarank.parameters():
  param.requires_grad = True

# Step 6: Training LambdaRank
optimizer = optim.Adam(lambdarank.parameters(), lr=1.0)  # Lowered LR, it should be like 0.1, 0.01, I've been experiencing gradient explosion
loss_fn = nn.BCEWithLogitsLoss()   # No Sigmoid Needed

# NDCG - based Gradient Scaling
def ndcg_gain(rank):
  return 1.0/np.log2(rank+2)   # NDCG Gain Formula

In [ ]:
# Debugging Gradient Updates
print("\n Checking Initial Weights and Biases")
for name, param in lambdarank.named_parameters():
    print(f"Before Training - {name}: {param.mean().item()}")


 Checking Initial Weights and Biases
Before Training - fc1.weight: 0.4799060523509979
Before Training - fc1.bias: 0.0
Before Training - fc2.weight: 0.22811557352542877
Before Training - fc2.bias: 0.0
Before Training - fc3.weight: -0.3348524272441864
Before Training - fc3.bias: 0.0


In [ ]:
# Training Loop
for epoch in range(50):
    total_loss = 0
    for a, b, label in pairwise_data:
        # I should be giving the entire list rather than pairwise
        input_a = torch.tensor([fasttext_df.loc[int(a), 'normalized_fit'] + 1e-7]).float().unsqueeze(0).unsqueeze(1)
        input_b = torch.tensor([fasttext_df.loc[int(b), 'normalized_fit'] + 1e-7]).float().unsqueeze(0).unsqueeze(1)

        output_a = lambdarank(input_a)  # I tried to train the rank like RankNet, and this part should also be modified
        output_b = lambdarank(input_b)
        # It's hard to print out the neural network
        # print out the loss, when debugging the model


        pred = (output_a - output_b).clamp(min=1e-7, max=1-1e-7)
        target = torch.tensor([label], dtype=torch.float32)

        loss = loss_fn(pred.view(-1), target.view(-1))
        total_loss += loss.item()

        # print out the losses to check whether the model is well-training (ex, gradient explosion or vanishing gradients etc..)

        optimizer.zero_grad()   # <-- should be out of the for loop, or epochs, It's zeroing the gradient and should not be here, maybe outside the forloops or either before epochs
        # before calling the model, inside the forloop
        loss.backward(retain_graph=True)  # Retain Graph for Debugging

        # # Print Gradients to Check If They Are Updating
        # for name, param in lambdarank.named_parameters():
        #     if param.grad is not None:
        #         print(f"Gradients at Epoch {epoch} - {name}: {param.grad.mean().item()}")

        # Clip gradients to prevent vanishing updates
        torch.nn.utils.clip_grad_norm_(lambdarank.parameters(), max_norm=1)

        optimizer.step()

    # Print loss every 50 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss}")

    # Debug: Stop if loss is not changing
    # if total_loss == 0 or torch.isnan(torch.tensor(total_loss)):
    #     print(f" Model is NOT learning! Check gradients & loss function.")
    #     break

    # Check if model weights are changing every 50 epochs
    if epoch % 10 == 0:
        for name, param in lambdarank.named_parameters():
            print(f"After Epoch {epoch} - {name}: {param.data.mean().item()}")

Epoch 0, Loss: 990.8020888566971
After Epoch 0 - fc1.weight: 0.4799060523509979
After Epoch 0 - fc1.bias: 0.0
After Epoch 0 - fc2.weight: 0.22811557352542877
After Epoch 0 - fc2.bias: 0.0
After Epoch 0 - fc3.weight: -0.3348524272441864
After Epoch 0 - fc3.bias: 0.0
Epoch 10, Loss: 990.8020888566971
After Epoch 10 - fc1.weight: 0.4799060523509979
After Epoch 10 - fc1.bias: 0.0
After Epoch 10 - fc2.weight: 0.22811557352542877
After Epoch 10 - fc2.bias: 0.0
After Epoch 10 - fc3.weight: -0.3348524272441864
After Epoch 10 - fc3.bias: 0.0
Epoch 20, Loss: 990.8020888566971
After Epoch 20 - fc1.weight: 0.4799060523509979
After Epoch 20 - fc1.bias: 0.0
After Epoch 20 - fc2.weight: 0.22811557352542877
After Epoch 20 - fc2.bias: 0.0
After Epoch 20 - fc3.weight: -0.3348524272441864
After Epoch 20 - fc3.bias: 0.0
Epoch 30, Loss: 990.8020888566971
After Epoch 30 - fc1.weight: 0.4799060523509979
After Epoch 30 - fc1.bias: 0.0
After Epoch 30 - fc2.weight: 0.22811557352542877
After Epoch 30 - fc2.bias:

In [ ]:
# **Step 7: Ranking Candidates**
def rank_candidates(df, model):
    predictions = []
    for i in range(len(df)):
        input_tensor = torch.tensor([df.loc[i, 'normalized_fit'] + 1e-7]).float().unsqueeze(0).unsqueeze(1)
        score = model(input_tensor).item()

        # Apply sigmoid to normalize scores between 0-1
        normalized_score = torch.sigmoid(torch.tensor(score)).item()

        predictions.append((df.loc[i, 'job_title'], normalized_score))

    # Sort by predicted score
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions

# **Final Ranking**
ranked_candidates = rank_candidates(fasttext_df, lambdarank)
print("\nFinal Ranking:")
for rank, (title, score) in enumerate(ranked_candidates, 1):
    print(f"{rank}. {title} (Score: {score:.4f})")


Final Ranking:
1. Aspiring Human Resources Professional (Score: 0.4993)
2. Aspiring Human Resources Professional (Score: 0.4993)
3. Human Resources Coordinator at InterContinental Buckhead Atlanta (Score: 0.4993)
4. People Development Coordinator at Ryan (Score: 0.4993)
5. Nortia Staffing is seeking Human Resources, Payroll & Administrative Professionals!!  (408) 709-2621 (Score: 0.4993)
6. Seeking Human Resources HRIS and Generalist Positions (Score: 0.0000)
7. HR Senior Specialist (Score: 0.0000)
8. Human Resources Management Major (Score: 0.0000)
9. Human Resources Specialist at Luxottica (Score: 0.0000)
10. Information Systems Specialist and Programmer with a love for data and organization. (Score: 0.0000)
11. HR Manager at Endemol Shine North America (Score: 0.0000)
12. Undergraduate Research Assistant at Styczynski Lab (Score: 0.0000)
13. Seeking employment opportunities within Customer Service or Patient Care (Score: 0.0000)
14. RRP Brand Portfolio Executive at JTI (Japan Tobac